# 字母识别-模型训练
## 导入所需模块

In [ ]:
import numpy as np
# import cv2
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
import net
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 数据预处理
加载手写数字数据集并进行图片预处理

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

# 加载数据
trainset = torchvision.datasets.EMNIST(root='../../../models/local/datasets/emnist',train=True, download=False, transform=transform, split='letters')

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=False,num_workers=0)


testset = torchvision.datasets.EMNIST(root='../../../models/local/datasets/emnist',train=False,download=False,transform=transform, split='letters')
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False,num_workers=0)

print(trainset.train_data[0])

classes = [ '' ,'a', 'b', 'c',  'd',  'e',  'f',
    'g',  'h', 'i',  'j', 'k','l', 'm', 'n','o', 'p', 'q', 'r', 's', 't', 'u', 'v','w', 'x', 'y',  'z',]

plt.imshow(trainset.train_data[0].numpy(), cmap='gray')
plt.title('label is {}'.format(classes[int(trainset.train_labels[0])]))
plt.show()


## 参数设置

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#net
net = net.Net()
net.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

num_epochs = 300 #训练次数

## 定义模型训练函数

In [ ]:
def train():
    print("开始训练")
    correct_preds = 0
    for epoch in range(num_epochs):
        net.train()
        for i, data in enumerate(trainloader):
            inputs, labels = data
            labels -= 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            train_loss = criterion(outputs, labels)
            train_loss.backward()
            optimizer.step()
        scheduler.step()
        print('epoch：%d，训练数据集：loss:%.4f'%(epoch+1, train_loss.item()))
        net.eval()
        with torch.no_grad():
            # 在接下来的代码中，所有Tensor的requires_grad都会被设置为False
            correct = 0
            total = 0

            for data in testloader:
                images, labels = data
                labels -= 1
                images, labels = images.to(device), labels.to(device)
                out = net(images)
                valid_loss = criterion(out, labels)
                _, predicted = torch.max(out.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            correct_pre = 100 * correct / total

            print('测试数据集loss：    %.4f'%(valid_loss.item()))
            print('测试数据集准确率为：{}%'.format(correct_pre))  # 输出识别准确率

        # 保存训练模型
        if correct_pre > correct_preds:
            torch.save(net, 'students_models/EMNIST_student.pth')
            correct_preds = correct_pre
            print("模型已保存")

## 开始训练
由于该数据集图片数量较多，在dachbot上进行训练花费时间较长。
预计3分钟训练1个epoch。

In [ ]:
train()